<a href="https://colab.research.google.com/github/sureshmanem/Tensorflow_2.0/blob/master/DefectPredictionAlgoTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense, Bidirectional,LSTM, Lambda, Conv1D
import datetime
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [4]:
dfCase = pd.read_excel(r'https://github.com/sureshmanem/Tensorflow_2.0/blob/master/Case_cvs.xlsx?raw=true')

In [5]:
dfCase.head()

,Case: Case Number,Case: Case Status,Benefit Test Type,Benefit Test Count,Case: Client Commitment Date,Effective Date,Case: Date/Time Opened,Case: Submitted Date,Case: Subject,WA Owner Manager,Work Assignment: Owner Name,Case: Market Segment,Business Area,Case: Level of Difficulty,Case: Age (In Days) Calendar,Client Name,Case: Super Client,Work Assignment: Workflow Assignment Name,Case: Description,Case: # of Plans Affected,Case: Request Type,Case: Related to a Project,Work Assignment: Record Type,Priority,Work Assignment: Created By,Platform ID,Actual Hours of Effort,Work Assignment: Created Date,Work Assignment: Last Modified Date,Case: Case Date/Time Last Modified,Target Completion Date,Case: Testing Target Completion Date,Case: Met Coding TCD,Case: Met Testing TCD,Case: Met Client Commitment Date,Case: Date/Time Closed,Case: In Production Date,Actual End Date,Status,Case: Classification,Case: Source Type,Case: Case Origin
0,4317814,Closed,NaN,NaN,2019-07-01,2019-07-01,2019-05-16 15:19:00,2019-06-25 15:35:00,Baptist Homes Update,Samuel Cormaty,Suchendra Babu Penjuri,Health Plan,HPCST-COMM,Level 3,40,ProAct1,ProAct,QA Design - Baptist Homes Update,Removing co-payments and adding specialty Coin...,1.0,Client Created,0,Parallel Work Assignment,Normal,Michelle Jill,2996,1.0,2019-06-25,2019-06-25,2019-06-25 15:44:00,NaT,NaT,NaN,NaN,Yes,2019-06-25 15:44:00,2019-06-25,2019-06-25 15:38:00,Closed,Benefits,New Business Implementation,Email
1,4008180,Closed,NaN,NaN,2019-01-11,2019-01-01,2019-01-07 16:18:00,2019-01-07 16:18:00,CCO: Vicodin Remove DDL EGWP,Shalin Nath,Saranya Selvaraj,Health Plan,HPCST-MEDD,Level 3,2,MVP MEDICARE D HMO WEST,"MVP Health Plan, Inc.",QA Design - CCO: Vicodin Remove DDL EGWP,Please remove the DDL coding for Hydrocodone/A...,50.0,CCO-Clinical Benefits,0,Parallel Work Assignment,Urgent,Amanda Van Beekum,1501,1.0,2019-01-07,2019-01-09,2019-01-09 16:18:00,2019-01-08,2019-01-10,Yes,Yes,Yes,2019-01-09 16:18:00,2019-01-09,2019-01-09 13:00:00,Closed,CCO Clinical Benefits,Major Request,Email
2,4072187,Closed,NaN,NaN,2019-02-05,2019-02-01,2019-01-29 22:15:00,2019-01-29 22:15:00,MEDICA_MDHC42994,Samuel Cormaty,Jothi Priya Durairaj,Health Plan,HPCST-COMM,Level 3,8,COMM LARGE GROUP MIC MN,Medica Health Plans,QA Design - MEDICA_MDHC42994,MEDICA_MDHC42994,1.0,Benefit Builder,0,Parallel Work Assignment,Normal,Integration Benefit Builder,0281,2.0,2019-01-29,2019-01-31,2019-02-06 16:31:00,2019-01-31,2019-02-04,Yes,Yes,Yes,2019-02-06 16:31:00,2019-02-01,2019-01-31 10:36:00,Closed,Benefits,Minor Request,Email
3,4113046,Closed,NaN,NaN,2019-02-25,2019-03-01,2019-02-18 11:45:00,2019-02-18 11:45:00,CBC 3/1 Alt TF Update,Samuel Cormaty,Suman Punuru,Health Plan,HPCST-MEDD,Level 3,4,Capital BlueCross,Capital BlueCross,QA Design - CBC 3/1 Alt TF Update,Alt TF update Effective 3/1. Please run attach...,1.0,CCO-Clinical Benefits,0,Parallel Work Assignment,Normal,Aaron Peters,8600,1.0,2019-02-18,2019-02-21,2019-02-22 12:37:00,2019-02-19,2019-02-22,Yes,Yes,Yes,2019-02-22 12:37:00,2019-02-22,2019-02-21 09:14:00,Closed,CCO Clinical Benefits,Major Request,Email
4,4162007,Closed,NaN,NaN,2019-03-12,2019-02-28,2019-03-10 19:15:00,2019-03-10 19:15:00,NE Weekly Formulary update 02-28,Shalin Nath,Kameswararao Indrakanti,Health Plan,HPCST-MEDICAID,Level 3,4,WellCare Nebraska Medicaid,WellCare,QA Design - NE Weekly Formulary update 02-28,NE Weekly Formulary update 02-28 See CI for Co...,1.0,Drug Coverage,0,Parallel Work Assignment,Urgent,Julie Simmons,8896,2.0,2019-03-10,2019-03-12,2019-03-14 14:13:00,2019-03-11,2019-03-12,Yes,Yes,Yes,2019-03-14 14:12:00,2019-03-12,2019-03-12 13:30:00,Closed,Benefits,Minor Request,Email


In [7]:
dfCase['Business Area'].unique()

array(['HPCST-COMM', 'HPCST-MEDD', 'HPCST-MEDICAID', 'HPCST-EXCH',
       'HPCST-STCOB', 'HPCST-MEDD-DUALS', 'HPCST-BLUES', 'HPCST'],
      dtype=object)